In [23]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [24]:
import os
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import Model

In [25]:
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/Colab Notebooks/SkinDisease.zip', 'r') as zipObj:
   zipObj.extractall()

In [26]:
!pip install split_folders

In [27]:
import splitfolders
os.chdir('/content/Data')
splitfolders.ratio('/content/Data', output="/content/Output", seed=1337, ratio=(.75, 0.1,0.15))
os.chdir('../../')

Copying files: 3119 files [00:01, 2114.05 files/s]


In [36]:
train_dg = ImageDataGenerator(rescale=1/255, horizontal_flip=True)
validation_dg = ImageDataGenerator(rescale=1/255)
test_dg = ImageDataGenerator(rescale=1/255)

train_dir = '/content/Output/train'
validation_dir = '/content/Output/val'
test_dir = '/content/Output/test'

train_gen = train_dg.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
validation_gen = validation_dg.flow_from_directory(validation_dir, target_size=(224, 224), batch_size=10, class_mode='categorical')
test_gen = test_dg.flow_from_directory(test_dir, target_size=(224, 224), batch_size=10, class_mode='categorical',shuffle=False)

Found 2337 images belonging to 5 classes.
Found 309 images belonging to 5 classes.
Found 473 images belonging to 5 classes.


In [37]:
from tensorflow.keras.applications import ConvNeXtBase

pre_trained = ConvNeXtBase(input_shape=(224,224,3), include_top=False, weights='imagenet')

for layer in pre_trained.layers:
  layer.trainable=False

pre_trained.summary()

Model: "convnext_base"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10            │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_prestem_no… │ (None, 224, 224, 3)    │              0 │ input_layer_10[0][0]   │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stem        │ (None, 56, 56, 128)    │          6,528 │ convnext_base_prestem… │
│ (Sequential)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stage_0_bl… │ (None, 56, 56, 128)    │          6,400 │ convnext_base_stem[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stage_0_bl… │ (None, 56, 56, 128)    │            256 │ convnext_base_stage_0… │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stage_0_bl… │ (None, 56, 56, 512)    │         66,048 │ convnext_base_stage_0… │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stage_0_bl… │ (None, 56, 56, 512)    │              0 │ convnext_base_stage_0… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stage_0_bl… │ (None, 56, 56, 128)    │         65,664 │ convnext_base_stage_0… │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stage_0_bl… │ (None, 56, 56, 128)    │            128 │ convnext_base_stage_0… │
│ (LayerScale)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stage_0_bl… │ (None, 56, 56, 128)    │              0 │ convnext_base_stage_0… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_72 (Add)              │ (None, 56, 56, 128)    │              0 │ convnext_base_stem[0]… │
│                           │                        │                │ convnext_base_stage_0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stage_0_bl… │ (None, 56, 56, 128)    │          6,400 │ add_72[0][0]           │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stage_0_bl… │ (None, 56, 56, 128)    │            256 │ convnext_base_stage_0… │
│ (LayerNormalization)      │                        │                │                        │
├──────────────────────

 Total params: 87,566,464 (334.04 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 87,566,464 (334.04 MB)

In [39]:
last_layer = pre_trained.get_layer('layer_normalization_2')
last_output = last_layer.output
print(last_output)

<KerasTensor shape=(None, 7, 7, 1024), dtype=float32, sparse=False, name=keras_tensor_932>


In [40]:
x = Flatten()(last_output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(5, activation='softmax')(x)

model = Model(pre_trained.input, x)
model.summary()

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10            │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_prestem_no… │ (None, 224, 224, 3)    │              0 │ input_layer_10[0][0]   │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stem        │ (None, 56, 56, 128)    │          6,528 │ convnext_base_prestem… │
│ (Sequential)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stage_0_bl… │ (None, 56, 56, 128)    │          6,400 │ convnext_base_stem[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stage_0_bl… │ (None, 56, 56, 128)    │            256 │ convnext_base_stage_0… │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stage_0_bl… │ (None, 56, 56, 512)    │         66,048 │ convnext_base_stage_0… │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stage_0_bl… │ (None, 56, 56, 512)    │              0 │ convnext_base_stage_0… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stage_0_bl… │ (None, 56, 56, 128)    │         65,664 │ convnext_base_stage_0… │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stage_0_bl… │ (None, 56, 56, 128)    │            128 │ convnext_base_stage_0… │
│ (LayerScale)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stage_0_bl… │ (None, 56, 56, 128)    │              0 │ convnext_base_stage_0… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_72 (Add)              │ (None, 56, 56, 128)    │              0 │ convnext_base_stem[0]… │
│                           │                        │                │ convnext_base_stage_0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stage_0_bl… │ (None, 56, 56, 128)    │          6,400 │ add_72[0][0]           │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ convnext_base_stage_0_bl… │ (None, 56, 56, 128)    │            256 │ convnext_base_stage_0… │
│ (LayerNormalization)      │                        │                │                        │
├──────────────────────

 Total params: 138,952,837 (530.06 MB)

 Trainable params: 51,386,373 (196.02 MB)

 Non-trainable params: 87,566,464 (334.04 MB)

In [41]:
metrics = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)
history = model.fit(train_gen, epochs=2, validation_data=validation_gen, batch_size=10)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/2
74/74 ━━━━━━━━━━━━━━━━━━━━ 69s 596ms/step - accuracy: 0.2581 - loss: 31.7699 - precision_3: 0.2585 - recall_3: 0.2515 - val_accuracy: 0.4078 - val_loss: 1.4687 - val_precision_3: 0.8125 - val_recall_3: 0.0421
Epoch 2/2
74/74 ━━━━━━━━━━━━━━━━━━━━ 31s 411ms/step - accuracy: 0.3946 - loss: 1.4215 - precision_3: 0.5231 - recall_3: 0.1082 - val_accuracy: 0.4013 - val_loss: 1.4207 - val_precision_3: 0.6429 - val_recall_3: 0.0583


In [45]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

predictions = model.predict(test_gen)
predicted_classes = np.argmax(predictions, axis=1)

true_classes = test_gen.classes

conf_matrix = confusion_matrix(true_classes, predicted_classes)

print(conf_matrix)

print(classification_report(true_classes, predicted_classes))

48/48 ━━━━━━━━━━━━━━━━━━━━ 7s 138ms/step
[[ 19 103   0   0   0]
 [  6 164   2   0   0]
 [ 12  51   8   1   0]
 [ 10  56   0   0   0]
 [  2  38   1   0   0]]
              precision    recall  f1-score   support

           0       0.39      0.16      0.22       122
           1       0.40      0.95      0.56       172
           2       0.73      0.11      0.19        72
           3       0.00      0.00      0.00        66
           4       0.00      0.00      0.00        41

    accuracy                           0.40       473
   macro avg       0.30      0.24      0.20       473
weighted avg       0.36      0.40      0.29       473



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
